In [4]:
!pip install -q gradio pandas matplotlib

In [5]:
import requests
import pandas as pd
import gradio as gr
import matplotlib.pyplot as plt

BASE_URL = "https://server-cloud-v645.onrender.com/"

# --- Colors (hard-coded) ---
COLOR_TEMP = "#1f77b4"   # blue
COLOR_HUM  = "#ff7f0e"   # orange
COLOR_SOIL = "#2ca02c"   # green

STATUS_OK_COLOR = "#2ca02c"      # green
STATUS_WARN_COLOR = "#ffbf00"    # yellow
STATUS_BAD_COLOR = "#d62728"     # red


# ---------- Core Data Fetch ----------
def load_iot_data(feed: str, limit: int) -> pd.DataFrame | None:
    resp = requests.get(
        f"{BASE_URL}/history",
        params={"feed": feed, "limit": limit},
        timeout=30
    )
    data = resp.json()
    if "data" not in data or not data["data"]:
        return None

    df = pd.DataFrame(data["data"])
    if "created_at" not in df.columns or "value" not in df.columns:
        return None

    df["created_at"] = pd.to_datetime(df["created_at"], errors="coerce", utc=True)
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    df = df.dropna(subset=["created_at", "value"]).sort_values("created_at")

    return None if df.empty else df


# ---------- Helpers ----------
def normalize(series: pd.Series) -> pd.Series:
    mn, mx = float(series.min()), float(series.max())
    if mx - mn == 0:
        return series * 0.0
    return (series - mn) / (mx - mn)


# ---------- Plant Status + Plots ----------
def plant_dashboard(limit: int):
    try:
        dfs = {
            "temperature": load_iot_data("temperature", limit),
            "humidity": load_iot_data("humidity", limit),
            "soil": load_iot_data("soil", limit),
        }

        missing = [k for k, v in dfs.items() if v is None]
        if missing:
            return "⚠️ Partial Data", f"Missing sensors or empty history: {', '.join(missing)}", None, None, None, None

        temp = float(dfs["temperature"]["value"].iloc[-1])
        hum = float(dfs["humidity"]["value"].iloc[-1])
        soil = float(dfs["soil"]["value"].iloc[-1])

        issues, warnings = [], []

        checks = [
            ("Temperature", temp, 18, 32, 1),
            ("Air humidity", hum, 35, 75, 3),
            ("Soil moisture", soil, 20, 60, 3),
        ]

        for name, value, low, high, margin in checks:
            if not (low <= value <= high):
                issues.append(f"{name} out of range ({value:.1f})")
            elif value <= low + margin or value >= high - margin:
                warnings.append(f"{name} near limit ({value:.1f})")

        if issues:
            status = "🔴 Plant Status: Not OK"
            details_main = " ; ".join(issues)

        elif warnings:
            status = "🟡 Plant Status: Warning"
            details_main = " ; ".join(warnings)

        else:
            status = "🟢 Plant Status: OK"
            details_main = "All sensors are within valid ranges"

        details = (
    f"{details_main}\n"
    f"Latest values:\n"
    f"temp={temp:.1f}\n"
    f"humidity={hum:.1f}\n"
    f"soil={soil:.1f}"

        )

        df_t, df_h, df_s = dfs["temperature"], dfs["humidity"], dfs["soil"]

        fig_t = plt.figure(figsize=(7, 3.2))
        plt.plot(df_t["created_at"], df_t["value"], marker="o", color=COLOR_TEMP)
        plt.title("Temperature History")
        plt.xlabel("Time")
        plt.ylabel("°C")
        plt.grid(True)

        fig_h = plt.figure(figsize=(7, 3.2))
        plt.plot(df_h["created_at"], df_h["value"], marker="o", color=COLOR_HUM)
        plt.title("Air Humidity History")
        plt.xlabel("Time")
        plt.ylabel("%")
        plt.grid(True)

        fig_s = plt.figure(figsize=(7, 3.2))
        plt.plot(df_s["created_at"], df_s["value"], marker="o", color=COLOR_SOIL)
        plt.title("Soil Moisture History")
        plt.xlabel("Time")
        plt.ylabel("%")
        plt.grid(True)

        fig_c = plt.figure(figsize=(10, 3.4))
        plt.plot(df_t["created_at"], normalize(df_t["value"]), marker="o", label="Temperature (norm)", color=COLOR_TEMP)
        plt.plot(df_h["created_at"], normalize(df_h["value"]), marker="o", label="Humidity (norm)", color=COLOR_HUM)
        plt.plot(df_s["created_at"], normalize(df_s["value"]), marker="o", label="Soil (norm)", color=COLOR_SOIL)
        plt.title("Combined Trend (Normalized)")
        plt.xlabel("Time")
        plt.ylabel("Normalized Value (0–1)")
        plt.grid(True)
        plt.legend()

        return status, details, fig_t, fig_h, fig_s, fig_c

    except Exception:
        return "❌ Error", "Failed to fetch data from server. Please try again.", None, None, None, None


# ---------- Gradio UI (ONLY SCREEN 4: Plant Dashboard) ----------
with gr.Blocks() as demo:
    gr.HTML("""
<style>
  .legend-card {
    background-color: var(--background-fill-primary);
    color: var(--body-text-color);
  }

  .legend-card h4 {
    color: var(--body-text-color);
  }

.accent-btn button {
  background-color: var(--color-accent) !important;
  border-color: var(--color-accent) !important;
  color: #ffffff !important;
}

.accent-btn button:hover {
  filter: brightness(0.95);
}


</style>
""")


    gr.Markdown("""
<h1 style="text-align:center; font-size:36px; font-weight:700;">
🌱 CloudGarden – Plant Monitoring Dashboard
</h1>
""")
    gr.Markdown("""
<h2 style="text-align:center; font-size:26px; font-weight:600;">
🌿 Overall Plant Status (Real-Time)
</h2>
""")


    samples = gr.Slider(1, 200, value=20, step=1, label="Number of Samples (used for all graphs)")
    overall_btn = gr.Button("Update Plant Dashboard", variant="primary")



    overall_status = gr.Textbox(
        label="Overall Status",
        lines=1,
        placeholder="Click 'Update Plant Dashboard' to evaluate plant status"
    )
    overall_info = gr.Textbox(
        label="Status Details",
        lines=4,
        placeholder="Detailed plant analysis will appear here"
    )

    with gr.Row():
        gr.Markdown(f"""
<div class="legend-card" style="margin-top:14px;padding:14px;border:1px solid var(--border-color-primary)
;border-radius:10px;">




  <h4 style="margin-bottom:10px; font-size:20px; font-weight:600;">
🌿 Plant Status
</h4>


  <span style="color:{STATUS_OK_COLOR};font-size:26px;">●</span>
  <b>Healthy</b> – All sensor values within normal ranges<br>

  <span style="color:{STATUS_WARN_COLOR};font-size:26px;">●</span>
  <b>Warning</b> – At least one value near threshold<br>

  <span style="color:{STATUS_BAD_COLOR};font-size:26px;">●</span>
  <b>Not OK</b> – One or more values out of range<br><br>

  <span>Status is calculated automatically from sensor data</span>
</div>
        """)

        gr.Markdown(f"""
<div class="legend-card" style="margin-top:14px;padding:14px;border:1px solid var(--border-color-primary)
;border-radius:10px;">




  <h4 style="margin-bottom:10px; font-size:20px; font-weight:600;">
ℹ️ Valid Value Ranges
</h4>


  <span style="color:{COLOR_TEMP};font-size:26px;">●</span>
  🌡️ <b>Temperature</b>: 18–32°C<br>

  <span style="color:{COLOR_HUM};font-size:26px;">●</span>
  💧 <b>Air Humidity</b>: 35–75%<br>

  <span style="color:{COLOR_SOIL};font-size:26px;">●</span>
  🌱 <b>Soil Moisture</b>: 20–60%<br><br>

  <span>⚠️ Values outside these ranges are considered abnormal</span>
</div>
        """)

    gr.Markdown("""
<h2 style="text-align:center; margin-top:22px; font-size:26px; font-weight:600;">
📈 Plant Sensor Graphs
</h2>
""")

    with gr.Row():
        plot_temp = gr.Plot(label="Temperature")
        plot_hum = gr.Plot(label="Air Humidity")

    with gr.Row():
        plot_soil = gr.Plot(label="Soil Moisture")
        plot_combined = gr.Plot(label="Combined (Normalized)")

    overall_btn.click(
        fn=plant_dashboard,
        inputs=[samples],
        outputs=[overall_status, overall_info, plot_temp, plot_hum, plot_soil, plot_combined]
    )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://95ddb4705be3fbe345.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
